In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from EDA import get_test, get_train

In [93]:
X_train, y_train = get_train()
X_test = get_test()


In [94]:
print(X_train.shape)
# print(X_train.head(1))
print(X_test.shape)
print(X_test.dropna().shape)

(19168, 19)
(7000, 19)
(7000, 19)


In [95]:
# 训练随机森林解决回归问题
def random_forest_result(n, max_depth, min_samples_split):
    regressor = RandomForestRegressor(n_estimators=n, max_depth=max_depth, min_samples_split=min_samples_split)
    regressor.fit(X_train, y_train)
    return regressor.predict(X_test)

In [97]:
y_pred = random_forest_result(100, 20, min_samples_split=50)
print(y_pred.shape, y_pred)

(7000,) [1176906.50129997 1532643.02961006 1574419.1234109  ... 3259927.42679679
  802809.09843136 3544086.74855318]
